In [17]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import folium
from IPython.display import display

# 1. 데이터 불러오기
df = pd.read_csv("/home/gmlwls5168/code/gongmo/src/data/gwangjin_stat.csv")

# 2. 검색 주소 만들기 (도로명만 있을 경우 실패율 높음 → 구체화)
df["검색주소"] = "서울특별시 광진구 " + df["구간"].astype(str)

# 3. 지오코딩 준비
geolocator = Nominatim(user_agent="gwangjin_light_mapper")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# 4. 주소 → 위치 변환 (None 방지 위해 .notnull() 필터링 예정)
df["location"] = df["검색주소"].apply(geocode)

# 5. 지오코딩 실패 제거
df = df[df["location"].notnull()].copy()

# 6. 위도/경도 추출
df["위도"] = df["location"].apply(lambda loc: loc.latitude)
df["경도"] = df["location"].apply(lambda loc: loc.longitude)

# 7. 지도용 데이터 구성
map_df = df.dropna(subset=["위도", "경도"]).copy()
map_center = [map_df["위도"].mean(), map_df["경도"].mean()]

# 8. Folium 지도 생성
light_map = folium.Map(location=map_center, zoom_start=14)

# 9. 마커 추가
for _, row in map_df.iterrows():
    popup_text = f"""
    <b>구간:</b> {row['구간']}<br>
    <b>평균 조도:</b> {row['평균조도(LX)']} lx<br>
    <b>보행등:</b> {row['보행등 개수']}개<br>
    <b>차도등:</b> {row['차도등 개수']}개
    """
    folium.CircleMarker(
        location=[row["위도"], row["경도"]],
        radius=6,
        color="blue",
        fill=True,
        fill_opacity=0.6,
        popup=popup_text
    ).add_to(light_map)

# 10. Jupyter에서 바로 지도 표시
display(light_map)

# 💾 HTML로 저장하고 싶으면 아래 코드도 추가 가능
# light_map.save("/Users/heejin/code/gongmo/src/results/gwangjin_light_map.html")


ValueError: Location values cannot contain NaNs.